In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cudf as cd
import seaborn as sns
from plotly import express as px, graph_objects as go

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score as rocauc
import xgboost as xgb
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')
import gc
gc.enable()

In [ ]:
%%time
train = cd.read_csv('../input/xgboost-latest-please-do-it/train_new.csv').to_pandas()
test = cd.read_csv('../input/xgboost-latest-please-do-it/test_new.csv').to_pandas()
sample = cd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv').to_pandas()

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
%%time
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

In [ ]:
train

In [ ]:
test

In [ ]:
skf = StratifiedKFold(n_splits = 5,
                      shuffle = True,
                      random_state = 42)

In [ ]:
y = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv', usecols = ['target'], squeeze = True).astype(np.int8)
y

In [ ]:
scaler = StandardScaler()
param1 = {'n_estimators' : 12000,
                          'n_jobs' : -1,
                          'random_state' : 42,
                          'tree_method' : 'gpu_hist',
                          'learning_rate' : 1e-3,
        'colsample_bylevel': 0.7062958024220186, 
          'colsample_bynode': 0.4530555211336581, 
          'colsample_bytree': 0.5785681256096046, 
          'gamma': 6.688891313985078e-08, 
          'max_depth': 6, 
          'reg_alpha': 6.547277848299326e-07, 
          'reg_lambda': 6.299699953887313e-07, 
          'subsample': 0.3848784471351232}

param2 = {'n_estimators' : 12000,
                          'n_jobs' : -1,
                          'random_state' : 42,
                          'tree_method' : 'gpu_hist',
                          'learning_rate' : 1e-3,
    'colsample_bylevel': 0.1407950254506869,
          'colsample_bynode': 0.598924438784668, 
          'colsample_bytree': 0.18830911281395168, 
          'gamma': 0.0008420898648004424, 
          'max_depth': 7, 
          'reg_alpha': 1.3076491658824876e-06, 
          'reg_lambda': 4.79633597989686e-06, 
          'subsample': 0.3816336293270006}

param3 = {'n_estimators' : 12000,
                          'n_jobs' : -1,
                          'random_state' : 42,
                          'tree_method' : 'gpu_hist',
                          'learning_rate' : 1e-3,
    'colsample_bylevel': 0.3139676902866938, 
          'colsample_bynode': 0.4216069663162493,
          'colsample_bytree': 0.4225611422565575, 
          'gamma': 0.005695262371618228, 
          'max_depth': 8, 
          'reg_alpha': 2.2316175672387982e-07, 
          'reg_lambda': 0.00018606218311782073, 
          'subsample': 0.4447847723407572}

In [ ]:
scores1 = []
preds1 = []
val_preds1 = cd.DataFrame()
for i, (t, v) in enumerate(skf.split(train, y)) :
    x_train = train.iloc[t, :]
    x_val = train.iloc[v, :]
    x_test = test.copy()
    y_train = y[t]
    y_val = y[v]
    
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.transform(x_val)
    x_test = scaler.transform(x_test)
    
    model = XGBClassifier(**param1)
    model.fit(x_train, y_train,
              eval_set = [(x_val, y_val)],
              early_stopping_rounds = 250,
              eval_metric = 'auc',
              verbose = 1000)
    val_pred = model.predict_proba(x_val)[:, 1]
    pred = model.predict_proba(x_test)[:, 1]
    accuracy = rocauc(y_val, val_pred)
    print(i, ":", accuracy)
    scores1.append(accuracy)
    preds1.append(pred)
    del x_train
    del x_val
    del x_test
    del model
    del pred
    del val_pred
    del accuracy
    gc.collect()
mean_preds1 = np.mean(preds1, axis = 0)
med_preds1 = np.mean(preds1, axis = 0)

In [ ]:
scores2 = []
preds2 = []
val_preds2 = cd.DataFrame()
for i, (t, v) in enumerate(skf.split(train, y)) :
    x_train = train.iloc[t, :]
    x_val = train.iloc[v, :]
    x_test = test.copy()
    y_train = y[t]
    y_val = y[v]
    
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.transform(x_val)
    x_test = scaler.transform(x_test)
    
    model = XGBClassifier(**param2)
    model.fit(x_train, y_train,
              eval_set = [(x_val, y_val)],
              early_stopping_rounds = 250,
              eval_metric = 'auc',
              verbose = 1000)
    val_pred = model.predict_proba(x_val)[:, 1]
    pred = model.predict_proba(x_test)[:, 1]
    accuracy = rocauc(y_val, val_pred)
    print(i, ":", accuracy)
    scores2.append(accuracy)
    preds2.append(pred)
    del x_train
    del x_val
    del x_test
    del model
    del pred
    del val_pred
    del accuracy
    gc.collect()
mean_preds2 = np.mean(preds2, axis = 0)
med_preds2 = np.median(preds2, axis = 0)

In [ ]:
scores3 = []
preds3 = []
for i, (t, v) in enumerate(skf.split(train, y)) :
    x_train = train.iloc[t, :]
    x_val = train.iloc[v, :]
    x_test = test.copy()
    y_train = y[t]
    y_val = y[v]
    
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.transform(x_val)
    x_test = scaler.transform(x_test)
    
    model = XGBClassifier(**param3)
    model.fit(x_train, y_train,
              eval_set = [(x_val, y_val)],
              early_stopping_rounds = 250,
              eval_metric = 'auc',
              verbose = 1000)
    val_pred = model.predict_proba(x_val)[:, 1]
    pred = model.predict_proba(x_test)[:, 1]
    accuracy = rocauc(y_val, val_pred)
    print(i, ":", accuracy)
    scores3.append(accuracy)
    preds3.append(pred)
    del x_train
    del x_val
    del x_test
    del model
    del pred
    del val_pred
    del accuracy
    gc.collect()
mean_preds3 = np.mean(preds3, axis = 0)
med_preds3 = np.median(preds3, axis = 0)

In [ ]:
meanmean_preds = np.mean(np.column_stack((mean_preds1, mean_preds2, mean_preds3)), axis = 1)
sample['target'] = meanmean_preds
sample.to_csv('submission.csv', index = False)
sample

In [ ]:
meanmed_preds = np.median(np.column_stack((mean_preds1, mean_preds2, mean_preds3)), axis = 1)
sample['target'] = meanmed_preds
sample.to_csv('meanmedian_submission.csv', index = False)

In [ ]:
medmean_preds = np.mean(np.column_stack((med_preds1, med_preds2, med_preds3)), axis = 1)
sample['target'] = medmean_preds
sample.to_csv('medmean_submission.csv', index = False)

In [ ]:
medmed_preds = np.median(np.column_stack((med_preds1, med_preds2, med_preds3)), axis = 1)
sample['target'] = medmed_preds
sample.to_csv('medmed_submission.csv', index = False)